In [3]:

import json
import xmltodict
import pandas as pd
from urllib.parse import quote_plus, urlencode
from urllib.request import urlopen, Request

class AptDetailReader:

    URL = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev'
    NUM_OF_ROWS = "10000"
    def __init__(self, service_key):
        self.service_key = service_key
    
    def DataReader(self, LAWD_CD, DEAL_YMD):

        queryParams = '?' + urlencode({ \
            quote_plus('LAWD_CD') : LAWD_CD,
            quote_plus('DEAL_YMD') : DEAL_YMD,
            quote_plus('numOfRows') : self.NUM_OF_ROWS,
            })

        request = Request(self.URL + queryParams + self.service_key)
        request.get_method = lambda: 'GET'
        response_body = urlopen(request).read().decode('utf8')
        
        return self.__json2pd(xmltodict.parse(response_body))

    def __json2pd(self, json_data):

        items = json_data['response']['body']['items']
        if items is None: 
            return pd.DataFrame()

        datas = items['item']
        output = pd.DataFrame(columns=datas[0].keys())
        
        #pd.concat([ pd.DataFrame(data, columns=datas[0].keys()) for data in datas ], ignore_index=True)
        
        for data in datas :
            output = output.append(data, ignore_index=True)

        return output

In [4]:
apt = AptDetailReader("&ServiceKey=aKVcsSF8UlpRkppSsPS38IZO8UZ87fNBzMTtrrWtoadmn7ySBos%2BX8AOc6M%2F47Temd2cIbpl4%2BFxq%2Btu0KoKWA%3D%3D")
df_data = apt.DataReader("41135", "201911")
df_data.head()

,거래금액,건축년도,년,도로명,도로명건물본번호코드,도로명건물부번호코드,도로명시군구코드,도로명일련번호코드,도로명지상지하코드,도로명코드,...,법정동읍면동코드,법정동지번코드,아파트,월,일,일련번호,전용면적,지번,지역코드,층
0,"101,000",1992,2019,수내로,00201,00000,41135,01,0,3180039,...,10100,1,샛별마을(삼부),11,1,41135-20,128.09,39,41135,15
1,"85,000",1993,2019,장안로,00015,00000,41135,01,0,3180057,...,10100,1,장안타운(건영),11,2,41135-32,131.55,66,41135,15
2,"81,000",1992,2019,분당로,00190,00000,41135,01,0,3180026,...,10100,1,샛별마을(라이프),11,2,41135-19,84.99,34,41135,14
3,"84,800",1994,2019,수내로,00181,00000,41135,01,0,3180039,...,10100,1,샛별마을(우방),11,4,41135-21,84.99,38,41135,9
4,"82,000",1992,2019,분당로,00190,00000,41135,01,0,3180026,...,10100,1,샛별마을(라이프),11,4,41135-19,85,34,41135,9
